### Generate Cambdrige Data

This script leverages the CBECI API to download fresh Bitcoin energy consumption estimates from the [Cambridge Centre for Alternative Finance](https://ccaf.io/cbeci/index) and output the best guess annualized data for all energy price assumptions into a single CSV.

The motivation for this script is that the Cambridge Bitcoin Electricity Consumption Index assumes a constant fixed energy cost of $0.05/kWh, but in the face of rising energy prices there is desire to modify the output of the index to be more reflective of dynamic energy prices.

The output of this script is designed to be paired with EIA energy price data to allow for an evolving adjustment of what level of electricity spend the Bitcoin Network can support.

- Author: [Data Always](dataalways.substack.com)
- Last Modified: November 15, 2022

In [1]:
import numpy as np
import pandas as pd

In [2]:
"""
The analysis in this repository is only concerned with the CBECI
`BEST GUESS`. To avoid polution of the .csv we drop all the 
other columns. 
"""
def drop_standard_columns(dataframe):
    return dataframe.drop([
        'Timestamp', 
        'power MAX, GW', 
        'power MIN, GW', 
        'power GUESS, GW', 
        'annualised consumption MAX, TWh',
        'annualised consumption MIN, TWh'
        ], 
        axis= 1
    )

In [3]:
# Create the base dataframe on which to merge others by looking at the $0.01 data
df = pd.read_csv(
    "https://ccaf.io/cbeci/api/v1.2.0/download/data?price=0.01", 
    skiprows=1
)

# Drop the unwanted columns
df = drop_standard_columns(df)

# Rename the columns
df.rename(
    columns={
        'Date and Time': 'date', 
        'annualised consumption GUESS, TWh': 0.01
    }, 
    inplace=True
)

In [4]:
# Download the data for all prices and then merge them into one dataframe
for elec_cost in np.linspace(0.02, 0.19, 18):
    elec_str = "{:.2f}".format(elec_cost)
    
    tmp_df = pd.read_csv(
        f"https://ccaf.io/cbeci/api/v1.2.0/download/data?price={elec_str}", 
        skiprows=1
    )
    
    tmp_df = drop_standard_columns(tmp_df)
    
    tmp_df.rename(
        columns={
            'Date and Time': 'date', 
            'annualised consumption GUESS, TWh': elec_str
        }, 
        inplace=True
    )

    # Merge the data into the main data frame
    df = df.merge(
        tmp_df, 
        how="inner", 
        on="date"
    )

In [5]:
df.tail()

,date,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19
4527,2022-12-09T00:00:00,151.500234,136.716446,113.200122,102.546001,91.036452,83.545526,73.948546,70.241070,60.883710,54.269320,54.269320,52.661763,63.26991,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663
4528,2022-12-10T00:00:00,153.280306,138.322813,114.530182,103.750878,92.470026,84.833301,74.817414,71.317109,62.039485,54.906965,54.906965,52.661763,63.26991,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663
4529,2022-12-11T00:00:00,152.708335,137.806657,114.102808,103.363728,92.485496,84.810115,74.733667,71.303382,62.244037,54.702077,54.702077,52.661763,63.26991,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663
4530,2022-12-12T00:00:00,150.662203,135.960192,112.902351,102.199254,91.613097,83.966045,73.925053,70.607291,61.853539,53.969127,53.969127,52.661763,63.26991,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663
4531,2022-12-13T00:00:00,156.253981,139.710273,116.386312,105.514056,94.488456,86.585682,76.172803,72.671771,63.981552,55.457714,55.457714,53.457323,63.26991,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663


In [6]:
# Export the data to a .csv file
df.to_csv(
    './datasets/cbeci-annualized-consumption.csv', 
    index = False
)